# Initial experiments Part 2

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from dataset import QuickDrawDataset
from utils import AbsolutePenPositionTokenizer
from tqdm import tqdm
import pickle

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

seed = 42
torch.manual_seed(seed)
if device == "cuda":
    torch.cuda.manual_seed_all(seed)

Using device: cuda


In [11]:
labels = ["cat"]

training_data = QuickDrawDataset(
    labels=labels,
)

tokenizer = AbsolutePenPositionTokenizer(bins=64)

class SketchDataset(Dataset):
    def __init__(self, svg_list, tokenizer, max_len=200, cache_file="sketch_tokenized_dataset.pkl"):
        self.data = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad_id = tokenizer.vocab["PAD"]
        
        # Try to load from cache
        try:
            with open(cache_file, "rb") as f:
                self.data = pickle.load(f)
            print(f"Loaded tokenized data from {cache_file}")
        except FileNotFoundError:
            for svg in tqdm(svg_list, desc="Tokenizing SVGs"):
                tokens = tokenizer.encode(svg)
                # Truncate + Pad
                tokens = tokens[:max_len]
                tokens = tokens + [self.pad_id] * (max_len - len(tokens))
                self.data.append(tokens)
                
            # Save to cache
            with open(cache_file, "wb") as f:
                pickle.dump(self.data, f)
            print(f"Saved tokenized data to {cache_file}")

    def __getitem__(self, idx):
        seq = self.data[idx]
        input_ids = torch.tensor(seq[:-1])
        target_ids = torch.tensor(seq[1:])
        return input_ids, target_ids
    
    def __len__(self):
        return len(self.data)
    
dataset = SketchDataset(training_data, tokenizer, max_len=200)

Loading QuickDraw files: 100%|██████████| 1/1 [00:03<00:00,  3.32s/it]


Loaded tokenized data from sketch_tokenized_dataset.pkl


In [ ]:
def generate_square_subsequent_mask(sz: int):
    """Causal mask to stop attention to future positions"""
    return torch.triu(torch.ones(sz, sz), diagonal=1).bool()

class SketchTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=8, num_layers=6, max_len=200):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_len = max_len

        self.embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Embedding(max_len, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=4*d_model, activation='gelu'
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.norm = nn.LayerNorm(d_model)
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        """
        x: (batch, seq_len) input tokens
        Returns: (batch, seq_len, vocab_size) logits
        """
        batch_size, seq_len = x.shape
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)

        x = self.embed(x) + self.pos_embed(positions)  # (batch, seq_len, d_model)
        x = x.transpose(0, 1)  # -> (seq_len, batch, d_model)

        # causal mask (seq_len, seq_len)
        mask = generate_square_subsequent_mask(seq_len).to(x.device)

        x = self.transformer(x, mask=mask)  # (seq_len, batch, d_model)
        x = x.transpose(0, 1)  # back to (batch, seq_len, d_model)
        x = self.norm(x)

        logits = self.fc_out(x)  # (batch, seq_len, vocab_size)
        return logits
    
def train_model(model, dataloader, vocab_size, epochs=10, lr=1e-4, device="cuda"):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.vocab["PAD"]) # updated

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for input_ids, target_ids in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            input_ids, target_ids = input_ids.to(device), target_ids.to(device)

            # Transformer expects shape (seq_len, batch, d_model)
            logits = model(input_ids)  # (seq_len, batch, vocab_size)
            loss = criterion(logits.view(-1, vocab_size), target_ids.view(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss/len(dataloader):.4f}")


dataloader = DataLoader(dataset, batch_size=256, shuffle=True, pin_memory=True)
model = SketchTransformer(vocab_size=len(tokenizer.vocab), d_model=256, nhead=8, num_layers=6)

train_model(model, dataloader, vocab_size=len(tokenizer.vocab), epochs=40, lr=1e-3, device=device)

Epoch 1/40: 100%|██████████| 403/403 [01:33<00:00,  4.33it/s]


Epoch 1 Loss: 2.4070


Epoch 2/40: 100%|██████████| 403/403 [01:32<00:00,  4.34it/s]


Epoch 2 Loss: 1.7705


Epoch 3/40: 100%|██████████| 403/403 [01:33<00:00,  4.32it/s]


Epoch 3 Loss: 1.6398


Epoch 4/40: 100%|██████████| 403/403 [01:32<00:00,  4.34it/s]


Epoch 4 Loss: 1.5869


Epoch 5/40:  58%|█████▊    | 235/403 [00:54<00:38,  4.34it/s]

In [ ]:
# save the model
# torch.save(model, "sketch_transformer_cat_decoder_checkpoint.pth")

In [ ]:
# model = torch.load("sketch_transformer_cat_decoder_checkpoint.pth", map_location=device, weights_only=False)

def sample_sequence(model, start_token, max_len=200, temperature=1.0, greedy=False, eos_id=None, device="cuda"):
    model.eval()
    
    tokens = [start_token]
    tokens_tensor = torch.tensor([tokens], device=device)  # (1, 1)

    for _ in range(max_len - 1):
        with torch.no_grad():
            logits = model(tokens_tensor)  # (batch=1, seq_len, vocab_size)
            next_logits = logits[:, -1, :] / temperature  # take last step
            probs = F.softmax(next_logits, dim=-1)

            if greedy:
                next_token = torch.argmax(probs, dim=-1).item()
            else:
                next_token = torch.multinomial(probs, num_samples=1).item()

        tokens.append(next_token)

        # stop if EOS reached
        if eos_id is not None and next_token == eos_id:
            break

        next_token_tensor = torch.tensor([[next_token]], device=device)
        tokens_tensor = torch.cat([tokens_tensor, next_token_tensor], dim=1)

    return tokens

# TODO Top-k filtering, Top-p filtering (nucleus)

start_token = tokenizer.vocab["START"]
eos_token = tokenizer.vocab.get("END", None)

generated = sample_sequence(
    model, 
    start_token, 
    max_len=200, 
    temperature=0.5, 
    greedy=False, 
    eos_id=eos_token, 
    device=device
)

print("Generated token sequence:", generated)
decoded_sketch = tokenizer.decode(generated, stroke_width=0.4)
print("Decoded sketch:", decoded_sketch)

from IPython.display import HTML, display

display(HTML(f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated</b><br>{decoded_sketch}</div>'))

In [ ]:
def top_p_filtering(logits, p=0.9):
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

    sorted_indices_to_remove = cumulative_probs > p
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = False

    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[:, indices_to_remove] = -float("Inf")
    return logits

def top_k_filtering(logits, k):
    if k <= 0:
        return logits
    top_k = min(k, logits.size(-1))
    values, _ = torch.topk(logits, top_k)
    min_values = values[:, -1].unsqueeze(-1)
    logits[logits < min_values] = -float("Inf")
    return logits

def sample_sequence_feat(
    model, start_token, max_len=200, temperature=1.0,
    top_k=60, top_p=0.9,
    greedy=False, eos_id=None, device="cuda"
):
    model.eval()
    tokens = [start_token]
    tokens_tensor = torch.tensor([tokens], device=device)

    for _ in range(max_len - 1):
        with torch.no_grad():
            logits = model(tokens_tensor)
            next_logits = logits[:, -1, :] / temperature

            # top-k / top-p filtering
            next_logits = top_k_filtering(next_logits, top_k)
            next_logits = top_p_filtering(next_logits, top_p)

            probs = F.softmax(next_logits, dim=-1)

            if greedy:
                next_token = torch.argmax(probs, dim=-1).item()
            else:
                next_token = torch.multinomial(probs, num_samples=1).item()

        tokens.append(next_token)

        if eos_id is not None and next_token == eos_id:
            break

        next_token_tensor = torch.tensor([[next_token]], device=device)
        tokens_tensor = torch.cat([tokens_tensor, next_token_tensor], dim=1)

    return tokens


start_token = tokenizer.vocab["START"]
eos_token = tokenizer.vocab.get("END", None)

generated = sample_sequence_feat(
    model, 
    start_token, 
    max_len=200, 
    temperature=1.0, 
    greedy=False, 
    eos_id=eos_token, 
    device=device
)

print("Generated token sequence:", generated)
decoded_sketch = tokenizer.decode(generated, stroke_width=0.4)
print("Decoded sketch:", decoded_sketch)

from IPython.display import HTML, display

display(HTML(f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated</b><br>{decoded_sketch}</div>'))